<a href="https://colab.research.google.com/github/Andrewsas/trabalho-fundamentos-cid/blob/master/Modulo_4/Projeto_final_modulo_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports e Dependências

In [63]:
import pandas as pd
import requests

#Funções ETL

In [64]:
#Treat numeric values from dataframe
def str_to_num(df_aux, index):
  for i in index:
    col_name = df_aux.columns[i]
    df_aux[col_name] = df_aux[col_name].str.replace('.', '')
    df_aux[col_name] = df_aux[col_name].str.replace(',', '.')
    df_aux[col_name] = pd.to_numeric(df_aux[col_name])
  return df_aux


In [96]:
def get_data(url, id):
  url = (url)
  response = requests.get(url)
  print("Request: {} => {}".format(id, response))
  if response.status_code == 200:
    payload = response.json()
  else:
    print(f'Error {response.status_code}: {response.reason}')
  return payload

#Return dataframe movies/series from api informed
def retrieve_data_from_api(url, id):
  payload = get_data(url, id)
  return pd.DataFrame(payload["results"]), payload["page"], payload["total_pages"]

#Return dataframe cast (artists and directors) from api informed
def retrieve_data_cast_from_api(url, id):
  payload = get_data(url, id)
  print(payload["cast"])
  print(payload["crew"])
  return pd.DataFrame(payload["cast"]), pd.DataFrame(payload["crew"])

In [66]:
#Reference: https://www.themoviedb.org/documentation/api
BASE_URL = "https://api.themoviedb.org/3/"
API_KEY = "?api_key=c9fd270fb257dd3be3a6cc9eac4b81a6"
DEFAULT_LANGUAGE = "&language=en-US"
PAGE = "&page="
CAST_REQUEST = "/casts"
def build_themoviedb_url(request_type, page):
  uri = BASE_URL + request_type + API_KEY + DEFAULT_LANGUAGE 
  if page > 0:
    uri = uri + PAGE + str(page)
  return uri

#http://api.themoviedb.org/3/movie/550/casts?api_key=c9fd270fb257dd3be3a6cc9eac4b81a6
def build_themoviedb_cast_url(request_type, movie_id):
  uri = BASE_URL + request_type +  str(movie_id) + CAST_REQUEST + API_KEY
  return uri

In [116]:
def load_all_themoviedb(request):
  print(request)
  current_page = 0
  total_pages = 1
  while current_page <= total_pages:
    uri = build_themoviedb_url(request, current_page)
    df_aux, current_page, total_pages = retrieve_data_from_api(uri, current_page)
    if current_page == 1:
      print("TOTAL: ", total_pages)
      df = pd.DataFrame(df_aux)
    else:
      df = df.append(df_aux, ignore_index=True)
    current_page = current_page+1
  print("DONE!")
  return df

def load_all_cast_themoviedb(request, df_aux):
  for index, row in df_aux.iterrows():
    movie_id = row['id']
    uri = build_themoviedb_cast_url(request, movie_id)
    df_art, df_dir = retrieve_data_cast_from_api(uri, movie_id)
    #get Acting department
    artists = get_department(df_art, 'known_for_department', 'Acting')
    print(artists)
    #get Directing department
    directors = get_department(df_dir, 'department', 'Directing')
    print(directors)
    break
    
def get_department(df, column, department):
  if column in df:
    aux = df[df[column] == department]
  return aux['name'] 

In [68]:
def get_netflix_dataset():
  !wget https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/netflix_titles.csv?raw=true
  !ls -la
  #rename
  !mv 'netflix_titles.csv?raw=true' 'netflix_titles.csv'
  df = pd.read_csv('./netflix_titles.csv', sep = ",", usecols = ["type", "title", "director", "cast", "country", "date_added", "release_year", "rating", "duration", "listed_in", "description"])
  return df

##Métodos para checagem de **outliers**

In [69]:
def tukeys_method(df):
  # Parâmetros: series
  q1 = np.quantile(df, 0.25)
  q3 = np.quantile(df, 0.75)
  iqr = q3-q1
  inner_fence = 1.5*iqr
  outer_fence = 3*iqr

  #inner fence lower and uper end
  inner_fence_le = q1-inner_fence
  inner_fence_ue = q3+inner_fence

  #outer fence lower and uper end
  outer_fence_le = q1-outer_fence
  outer_fence_ue = q3+outer_fence

  outliers_prob = []
  outliers_poss = []
  for index, x in enumerate(df):
    if x <= outer_fence_le or x >= outer_fence_ue:
      outliers_prob.append(index)
  for index, x in enumerate(df):
    if x <= inner_fence_le or x >= inner_fence_ue:
      outliers_poss.append(index)
  return outliers_prob, outliers_poss

#Carregamento dos dados

1.   Filmes - [the movie database](https://www.themoviedb.org/)
2.   Série - [the movie database](https://www.themoviedb.org/)
3.   Netflix - [Kaggle](https://www.kaggle.com/shivamb/netflix-shows)

In [ ]:
#Movies
df_movies = load_all_themoviedb("movie/top_rated")
df_movies.info()

In [ ]:
#Series
df_series = load_all_themoviedb("tv/top_rated")
df_series.info()

In [ ]:
#Netflix
df_netflix = get_netflix_dataset()
df_netflix.info()

##Tratamento de Dados de filmes e séries
**Elenco:**
*   Atores
*   Diretores



In [117]:
#Movies
df_movies_cast = load_all_cast_themoviedb("movie/", df_movies)


Request: 761053 => <Response [200]>
[{'adult': False, 'gender': 2, 'id': 544002, 'known_for_department': 'Acting', 'name': 'Giulio Berruti', 'original_name': 'Giulio Berruti', 'popularity': 5.598, 'profile_path': '/ktPKniWGVkm6eBG7a2R7WGd96kZ.jpg', 'cast_id': 1, 'character': 'Gabriel Emerson', 'credit_id': '5fdad9cfeda4b70041400df3', 'order': 1}, {'adult': False, 'gender': 0, 'id': 2897282, 'known_for_department': 'Acting', 'name': 'Rhett Wellington', 'original_name': 'Rhett Wellington', 'popularity': 0.694, 'profile_path': None, 'cast_id': 2, 'character': 'Simon Talbot', 'credit_id': '5fdad9dd3f7e1d004042f859', 'order': 2}, {'adult': False, 'gender': 1, 'id': 1312450, 'known_for_department': 'Acting', 'name': 'Melanie Zanetti', 'original_name': 'Melanie Zanetti', 'popularity': 11.569, 'profile_path': '/lbUQ7ilvBtWMU23reKsHg3jRmsf.jpg', 'cast_id': 3, 'character': 'Julia Mitchell', 'credit_id': '5fdad9fa29c62600401f7a01', 'order': 3}, {'adult': False, 'gender': 0, 'id': 2913392, 'known_